In [1]:
SEED = 61


import os
import re
import gc
import h5py
import torch
import numpy as np
import pandas as pd
import tensorflow as tf
import random as python_random
import matplotlib.pyplot as plt
import tensorflow_addons as tfa

from tqdm import tqdm
from nltk import tokenize

from sklearn import preprocessing
from sklearn.decomposition import PCA
from keras import backend as K
from keras import initializers,regularizers,constraints
from keras.preprocessing.text import Tokenizer, text_to_word_sequence
from keras.preprocessing.sequence import pad_sequences
from keras.utils.np_utils import to_categorical
from keras.layers import Reshape, Input, Embedding, Flatten, Dense, Dropout, BatchNormalization, Activation #, merge
from keras.layers import TimeDistributed, LSTM, GRU, Bidirectional, Convolution1D, MaxPooling1D, MaxPooling2D
from keras.layers.core import RepeatVector, Reshape
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from keras.models import Sequential, Model, load_model
from sklearn.model_selection import cross_val_score

def reset_seeds():
    np.random.seed(SEED) 
    python_random.seed(SEED)
    tf.random.set_seed(SEED)
    os.environ["PYTHONHASHSEED"] = str(SEED)

# from tensorflow.python.keras.layers import Layer, InputSpec, Lambda
# from tensorflow.keras import Model
# from attention import Attention_input1, Attention_input2
# from keras.optimizers import SGD, RMSprop, Adagrad

In [2]:
def read_hdf5(path):
    read_file = h5py.File(path, 'r')
    
    feature_names = list(read_file.keys())
    loaded_data = []
    
    for name in feature_names:
        dataset = read_file[name][:]
        if dataset.dtype == np.dtype('object'):
            dataset = np.array([x.decode('UTF-8') for x in dataset])            
        loaded_data.append((name, dataset))

    return loaded_data

def loadz(path):
    data = np.load(path)['arr_0']
    return data

In [3]:
def merge_mvsa(mvsa_single, mvsa_multiple):
    mvsa = np.concatenate((mvsa_single, mvsa_multiple), axis=0)
    return mvsa

def load_mvsa_feature(feature_name, merge=False):
    folder_path = os.path.join('../input/mvsa-features/', feature_name)
    single_file = 'mvsa-single-{}.npz'.format(feature_name)
    multiple_file = 'mvsa-multiple-{}.npz'.format(feature_name)
    mvsa_single = loadz(os.path.join(folder_path, single_file))
    mvsa_multiple = loadz(os.path.join(folder_path, multiple_file))
    
    if merge == True:
        return merge_mvsa(mvsa_single, mvsa_multiple)
    
    return mvsa_single, mvsa_multiple

def load_labels(path):
    data = read_hdf5(path)

    for x in data:
        if x[0] == 'multimodal-labels':
            labels = x[1]
        if x[0] == 'text-labels':
            text_labels = x[1]
        if x[0] == 'image-labels':
            image_labels = x[1]
        
    return labels, text_labels, image_labels

In [4]:
def plot_metrics(history):
    fig = plt.figure(figsize=(20, 5))

    fig.add_subplot(1, 3, 1)
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('LOSS')
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['train', 'val'], loc='best')

    fig.add_subplot(1, 3, 2)
    plt.plot(history.history['accuracy'])
    plt.plot(history.history['val_accuracy'])
    plt.title('ACCURACY')
    plt.ylabel('accuracy')
    plt.xlabel('epoch')
    plt.legend(['train', 'val'], loc='best')

    fig.add_subplot(1, 3, 3)
    plt.plot(history.history['f1_score'])
    plt.plot(history.history['val_f1_score'])
    plt.title('F1-SCORE')
    plt.ylabel('f1-score')
    plt.xlabel('epoch')
    plt.legend(['train', 'val'], loc='best')

    plt.show()

In [5]:
# e.g. validation_split=0.1 -----> 8:1:1 ratio of train, val, test
def split_data(data, validation_split):
    num_val = int(validation_split * data.shape[0])
    data_train = data[:-(num_val*2)]
    data_val = data[-(num_val*2):-(num_val)]
    data_test = data[-num_val:]
    return data_train, data_val, data_test

In [6]:
def create_model_image(input_shape):
    f1_score = tfa.metrics.F1Score(num_classes=NUM_CLASSES, average='macro')
    
    image_input = Input(shape=input_shape)
    outputs = Dense(NUM_CLASSES, activation='softmax') (image_input)
    
    model = Model(image_input, outputs)
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy', f1_score]) # f1 #tf.keras.metrics.AUC()
    return model

In [7]:
def create_model_text(input_shape):
    f1_score = tfa.metrics.F1Score(num_classes=NUM_CLASSES, average='macro')
    
    text_input = Input(shape=input_shape)
    reshape_text = Reshape((1, -1)) (text_input)
    lstm = LSTM(NUM_LSTM) (reshape_text)
    outputs = Dense(NUM_CLASSES, activation='softmax') (lstm)
    
    model = Model(text_input, outputs)
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy', f1_score]) # f1 #tf.keras.metrics.AUC()
    
    return model

In [8]:
def evaluate_model(model, X_test, y_test, checkpoint=None, verbose=1):
    if checkpoint is not None:
        model = load_model('./model_checkpoint/{}.h5'.format(checkpoint))#, custom_objects={'f1': f1})
    
    loss, acc, f1 = model.evaluate(X_test, y_test, verbose=verbose)
    
    if verbose == 1:
        print('Loss:', loss)
        print('Accuracy:', acc)
        print('F1-score:', f1)
        
    return loss, acc, f1

In [9]:
def run_and_evaluate(name, X, y, verbose=0):
    y = le.fit_transform(y)
    y = to_categorical(np.asarray(y))
    
    X_train, X_val, X_test = split_data(X, VALIDATION_SPLIT)
    y_train, y_val, y_test = split_data(y, VALIDATION_SPLIT)
    
    model = create_model_text(X_train.shape[1:])
    early_stopping = EarlyStopping(monitor='val_loss', min_delta=1e-4, patience=EARLY_STOPPING)
    checkpoint = ModelCheckpoint('./model_checkpoint/{}.h5'.format(name), save_best_only=True, verbose=verbose)
    
    history = model.fit(X_train, y_train, validation_data=(X_val, y_val), 
                        epochs=EPOCHS, batch_size=BATCH_SIZE, verbose=verbose,
                        callbacks=[checkpoint, early_stopping])
    
    best_epoch = np.argmin(history.history['val_loss'])
    print('Checkpoint loaded at epoch:', best_epoch)
    
    return history, evaluate_model(model, X_test, y_test, checkpoint=name, verbose=verbose)

In [10]:
def style_dataframe(dataframe):
    return dataframe.style.highlight_max(subset=['Accuracy', 'F1-score'], props='color:chartreuse', axis=0)\
                          .highlight_min(subset=['Accuracy', 'F1-score'], props='color:tomato', axis=0)\
                          .highlight_min(subset=['Loss'], props='color:chartreuse', axis=0)\
                          .highlight_max(subset=['Loss'], props='color:tomato', axis=0)

In [11]:
from IPython.display import display_html

def display_dataframes(dfs, names=[], index=False):
    def to_df(x):
        if isinstance(x, pd.Series):
            return pd.DataFrame(x)
        else:
            return x
    html_str = ''
    if names:
        html_str += ('<tr>' + 
                     ''.join(f'<td style="text-align:center">{name}</td>' for name in names) + 
                     '</tr>')
    html_str += ('<tr>' + 
                 ''.join(f'<td style="vertical-align:top"> {to_df(df).to_html()}</td>' 
                         for df in dfs) + 
                 '</tr>')
    html_str = f'<table>{html_str}</table>'
    html_str = html_str.replace('table','table style="display:inline"')
    display_html(html_str, raw=True)

# Load data

In [12]:
mvsa_single_bert_base, mvsa_multiple_bert_base = load_mvsa_feature('bert-base')
mvsa_single_pos_bow, mvsa_multiple_pos_bow = load_mvsa_feature('pos-bow')
mvsa_single_pos_tfidf, mvsa_multiple_pos_tfidf = load_mvsa_feature('pos-tfidf')
mvsa_single_ner_bow, mvsa_multiple_ner_bow = load_mvsa_feature('ner-bow')
mvsa_single_ner_tfidf, mvsa_multiple_ner_tfidf = load_mvsa_feature('ner-tfidf')

In [13]:
mvsa_bert_base = load_mvsa_feature('bert-base', merge=True)
mvsa_pos_bow = load_mvsa_feature('pos-bow', merge=True)
mvsa_pos_tfidf = load_mvsa_feature('pos-tfidf', merge=True)
mvsa_ner_bow = load_mvsa_feature('ner-bow', merge=True)
mvsa_ner_tfidf = load_mvsa_feature('ner-tfidf', merge=True)

In [14]:
mvsa_single_multimodal_labels, mvsa_single_text_labels, mvsa_single_image_labels = load_labels('../input/mvsa-features/labels/mvsa-single-labels.hdf5')
mvsa_multiple_multimodal_labels, mvsa_multiple_text_labels, mvsa_multiple_image_labels = load_labels('../input/mvsa-features/labels/mvsa-multiple-labels.hdf5')

mvsa_multimodal_labels = merge_mvsa(mvsa_single_multimodal_labels, mvsa_multiple_multimodal_labels)
mvsa_text_labels = merge_mvsa(mvsa_single_text_labels, mvsa_multiple_text_labels)
mvsa_image_labels = merge_mvsa(mvsa_single_image_labels, mvsa_multiple_image_labels)

In [15]:
le = preprocessing.LabelEncoder()
le.fit(mvsa_multimodal_labels)
NUM_CLASSES = len(le.classes_) # =3

In [16]:
feature_names = ['bert-base', 'bert-pos-bow', 'bert-pos-ner-bow', 'bert-pos-tfidf', 'bert-pos-ner-tfidf']

mvsa_single_features = [mvsa_single_bert_base, 
                        np.concatenate((mvsa_single_bert_base, mvsa_single_pos_bow), axis=1), 
                        np.concatenate((mvsa_single_bert_base, mvsa_single_pos_bow, mvsa_single_ner_bow), axis=1),
                        np.concatenate((mvsa_single_bert_base, mvsa_single_pos_tfidf), axis=1), 
                        np.concatenate((mvsa_single_bert_base, mvsa_single_pos_tfidf, mvsa_single_ner_tfidf), axis=1)]

mvsa_multiple_features = [mvsa_multiple_bert_base, 
                          np.concatenate((mvsa_multiple_bert_base, mvsa_multiple_pos_bow), axis=1), 
                          np.concatenate((mvsa_multiple_bert_base, mvsa_multiple_pos_bow, mvsa_multiple_ner_bow), axis=1),
                          np.concatenate((mvsa_multiple_bert_base, mvsa_multiple_pos_tfidf), axis=1), 
                          np.concatenate((mvsa_multiple_bert_base, mvsa_multiple_pos_tfidf, mvsa_multiple_ner_tfidf), axis=1)]

mvsa_features = [mvsa_bert_base,
                 np.concatenate((mvsa_bert_base, mvsa_pos_bow), axis=1), 
                 np.concatenate((mvsa_bert_base, mvsa_pos_bow, mvsa_ner_bow), axis=1),
                 np.concatenate((mvsa_bert_base, mvsa_pos_tfidf), axis=1), 
                 np.concatenate((mvsa_bert_base, mvsa_pos_tfidf, mvsa_ner_bow), axis=1)]

# Run models and Evalution display

In [17]:
reset_seeds()
EPOCHS = 100
BATCH_SIZE = 128
VALIDATION_SPLIT = 0.1
EARLY_STOPPING = 10
NUM_LSTM = 128

## With original text labels

In [18]:
print('MVSA-Single: With original text labels\n')
# run model with all features data of MVSA-Single
mvsa_single_hitories = []
mvsa_single_score = []
for i in range(len(feature_names)):
    print('MVSA-Single:', feature_names[i])
    history, scores = run_and_evaluate('single-' + feature_names[i], mvsa_single_features[i], mvsa_single_text_labels, verbose=0)
    mvsa_single_hitories.append(history)
    mvsa_single_score.append(scores)
    print()
    
df_single_scores = pd.DataFrame(mvsa_single_score, columns=['Loss', 'Accuracy', 'F1-score'], index=feature_names)

MVSA-Single: With original text labels

MVSA-Single: bert-base


2022-06-30 20:38:18.400384: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-06-30 20:38:18.401413: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-06-30 20:38:18.402099: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-06-30 20:38:18.402940: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compil

Checkpoint loaded at epoch: 6

MVSA-Single: bert-pos-bow
Checkpoint loaded at epoch: 6

MVSA-Single: bert-pos-ner-bow
Checkpoint loaded at epoch: 6

MVSA-Single: bert-pos-tfidf
Checkpoint loaded at epoch: 6

MVSA-Single: bert-pos-ner-tfidf
Checkpoint loaded at epoch: 6



In [19]:
print('MVSA-Multiple: With original image labels\n')
# run model with all features data of MVSA-Multiple
mvsa_multiple_hitories = []
mvsa_multiple_score = []
for i in range(len(feature_names)):
    print('MVSA-Multiple:', feature_names[i])
    history, scores = run_and_evaluate('multiple-' + feature_names[i], mvsa_multiple_features[i], mvsa_multiple_text_labels, verbose=0)
    mvsa_multiple_hitories.append(history)
    mvsa_multiple_score.append(scores)
    print()
    
df_multiple_scores = pd.DataFrame(mvsa_multiple_score, columns=['Loss', 'Accuracy', 'F1-score'], index=feature_names)

MVSA-Multiple: With original image labels

MVSA-Multiple: bert-base
Checkpoint loaded at epoch: 3

MVSA-Multiple: bert-pos-bow
Checkpoint loaded at epoch: 3

MVSA-Multiple: bert-pos-ner-bow
Checkpoint loaded at epoch: 3

MVSA-Multiple: bert-pos-tfidf
Checkpoint loaded at epoch: 3

MVSA-Multiple: bert-pos-ner-tfidf
Checkpoint loaded at epoch: 3



In [20]:
mvsa_average_scores = np.mean([mvsa_single_score, mvsa_multiple_score], axis=0)
df_average_scores = pd.DataFrame(mvsa_average_scores, columns=['Loss', 'Accuracy', 'F1-score'], index=feature_names)

In [21]:
print('With original image labels\n')
display_dataframes((style_dataframe(df_single_scores), style_dataframe(df_multiple_scores), style_dataframe(df_average_scores)), 
                   names=['MVSA-Single', 'MVSA-Multiple', 'Average'])

With original image labels



MVSA-Single MVSA-Multiple Average 
 
 
 
   
 Loss 
 Accuracy 
 F1-score 
 
 
 
 
 bert-base 
 0.721703 
 0.707657 
 0.675780 
 
 
 bert-pos-bow 
 0.728056 
 0.700696 
 0.669886 
 
 
 bert-pos-ner-bow 
 0.733448 
 0.689095 
 0.657411 
 
 
 bert-pos-tfidf 
 0.721896 
 0.705336 
 0.664209 
 
 
 bert-pos-ner-tfidf 
 0.737369 
 0.698376 
 0.665181 
 
 
 
 
 
 
 
   
 Loss 
 Accuracy 
 F1-score 
 
 
 
 
 bert-base 
 0.804546 
 0.597680 
 0.540417 
 
 
 bert-pos-bow 
 0.804690 
 0.607448 
 0.549103 
 
 
 bert-pos-ner-bow 
 0.805016 
 0.602564 
 0.544780 
 
 
 bert-pos-tfidf 
 0.803599 
 0.602564 
 0.537163 
 
 
 bert-pos-ner-tfidf 
 0.808248 
 0.602564 
 0.546119 
 
 
 
 
 
 
 
   
 Loss 
 Accuracy 
 F1-score 
 
 
 
 
 bert-base 
 0.763124 
 0.652668 
 0.608099 
 
 
 bert-pos-bow 
 0.766373 
 0.654072 
 0.609494 
 
 
 bert-pos-ner-bow 
 0.769232 
 0.645830 
 0.601096 
 
 
 bert-pos-tfidf 
 0.762747 
 0.653950 
 0.600686 
 
 
 bert-pos-ner-tfidf 
 0.772808 
 0.650470 
 0.605650

## With multimodal labels

In [22]:
print('With multimodal labels')

# run model with all features data of MVSA-Single
mvsa_single_hitories2 = []
mvsa_single_score2 = []
for i in range(len(feature_names)):
    print('MVSA-Single:', feature_names[i])
    history, scores = run_and_evaluate('single-' + feature_names[i], mvsa_single_features[i], mvsa_single_multimodal_labels, verbose=0)
    mvsa_single_hitories2.append(history)
    mvsa_single_score2.append(scores)
    print()
    
df_single_scores2 = pd.DataFrame(mvsa_single_score2, columns=['Loss', 'Accuracy', 'F1-score'], index=feature_names)

With multimodal labels
MVSA-Single: bert-base
Checkpoint loaded at epoch: 3

MVSA-Single: bert-pos-bow
Checkpoint loaded at epoch: 3

MVSA-Single: bert-pos-ner-bow
Checkpoint loaded at epoch: 3

MVSA-Single: bert-pos-tfidf
Checkpoint loaded at epoch: 3

MVSA-Single: bert-pos-ner-tfidf
Checkpoint loaded at epoch: 3



In [23]:
print('With multimodal labels')
# run model with all features data of MVSA-Multiple
mvsa_multiple_hitories2 = []
mvsa_multiple_score2 = []
for i in range(len(feature_names)):
    print('MVSA-Multiple:', feature_names[i])
    history, scores = run_and_evaluate('multiple-' + feature_names[i], mvsa_multiple_features[i], mvsa_multiple_multimodal_labels, verbose=0)
    mvsa_multiple_hitories2.append(history)
    mvsa_multiple_score2.append(scores)
    print()
    
df_multiple_scores2 = pd.DataFrame(mvsa_multiple_score2, columns=['Loss', 'Accuracy', 'F1-score'], index=feature_names)

With multimodal labels
MVSA-Multiple: bert-base
Checkpoint loaded at epoch: 2

MVSA-Multiple: bert-pos-bow
Checkpoint loaded at epoch: 2

MVSA-Multiple: bert-pos-ner-bow
Checkpoint loaded at epoch: 2

MVSA-Multiple: bert-pos-tfidf
Checkpoint loaded at epoch: 2

MVSA-Multiple: bert-pos-ner-tfidf
Checkpoint loaded at epoch: 2



In [24]:
mvsa_average_scores2 = np.mean([mvsa_single_score2, mvsa_multiple_score2], axis=0)
df_average_scores2 = pd.DataFrame(mvsa_average_scores2, columns=['Loss', 'Accuracy', 'F1-score'], index=feature_names)

In [25]:
print('With multimodal labels\n')
display_dataframes((style_dataframe(df_single_scores2), style_dataframe(df_multiple_scores2), style_dataframe(df_average_scores2)), 
                   names=['MVSA-Single', 'MVSA-Multiple', 'Average'])

With multimodal labels



MVSA-Single MVSA-Multiple Average 
 
 
 
   
 Loss 
 Accuracy 
 F1-score 
 
 
 
 
 bert-base 
 0.752630 
 0.675174 
 0.506405 
 
 
 bert-pos-bow 
 0.747594 
 0.691415 
 0.528983 
 
 
 bert-pos-ner-bow 
 0.740808 
 0.670534 
 0.509684 
 
 
 bert-pos-tfidf 
 0.748026 
 0.677494 
 0.518530 
 
 
 bert-pos-ner-tfidf 
 0.717518 
 0.686775 
 0.508679 
 
 
 
 
 
 
 
   
 Loss 
 Accuracy 
 F1-score 
 
 
 
 
 bert-base 
 0.772615 
 0.654457 
 0.438362 
 
 
 bert-pos-bow 
 0.771116 
 0.657509 
 0.441000 
 
 
 bert-pos-ner-bow 
 0.770156 
 0.658120 
 0.439741 
 
 
 bert-pos-tfidf 
 0.772120 
 0.662393 
 0.452794 
 
 
 bert-pos-ner-tfidf 
 0.773101 
 0.661783 
 0.448350 
 
 
 
 
 
 
 
   
 Loss 
 Accuracy 
 F1-score 
 
 
 
 
 bert-base 
 0.762623 
 0.664815 
 0.472383 
 
 
 bert-pos-bow 
 0.759355 
 0.674462 
 0.484991 
 
 
 bert-pos-ner-bow 
 0.755482 
 0.664327 
 0.474712 
 
 
 bert-pos-tfidf 
 0.760073 
 0.669944 
 0.485662 
 
 
 bert-pos-ner-tfidf 
 0.745310 
 0.674279 
 0.478514

# With merge MVSA data

In [26]:
print('With original image labels')
mvsa_hitories3 = []
mvsa_score3 = []
for i in range(len(feature_names)):
    print('Both MVSA:', feature_names[i])
    history, scores = run_and_evaluate('single-' + feature_names[i], mvsa_features[i], mvsa_image_labels, verbose=0)
    mvsa_hitories3.append(history)
    mvsa_score3.append(scores)
    print()
    
df_scores3 = pd.DataFrame(mvsa_score3, columns=['Loss', 'Accuracy', 'F1-score'], index=feature_names)

With original image labels
Both MVSA: bert-base
Checkpoint loaded at epoch: 3

Both MVSA: bert-pos-bow
Checkpoint loaded at epoch: 1

Both MVSA: bert-pos-ner-bow
Checkpoint loaded at epoch: 1

Both MVSA: bert-pos-tfidf
Checkpoint loaded at epoch: 1

Both MVSA: bert-pos-ner-tfidf
Checkpoint loaded at epoch: 1



In [27]:
print('With multimodal labels')
mvsa_hitories4 = []
mvsa_score4 = []
for i in range(len(feature_names)):
    print('Both MVSA:', feature_names[i])
    history, scores = run_and_evaluate('single-' + feature_names[i], mvsa_features[i], mvsa_multimodal_labels, verbose=0)
    mvsa_hitories4.append(history)
    mvsa_score4.append(scores)
    print()
    
df_scores4 = pd.DataFrame(mvsa_score4, columns=['Loss', 'Accuracy', 'F1-score'], index=feature_names)

With multimodal labels
Both MVSA: bert-base
Checkpoint loaded at epoch: 3

Both MVSA: bert-pos-bow
Checkpoint loaded at epoch: 1

Both MVSA: bert-pos-ner-bow
Checkpoint loaded at epoch: 1

Both MVSA: bert-pos-tfidf
Checkpoint loaded at epoch: 3

Both MVSA: bert-pos-ner-tfidf
Checkpoint loaded at epoch: 3



In [28]:
print('With both MVSA merged together\n')
display_dataframes((style_dataframe(df_scores3), style_dataframe(df_scores4)), 
                   names=['Original image labels', 'Multimodal labels'])

With both MVSA merged together



Original image labels Multimodal labels 
 
 
 
   
 Loss 
 Accuracy 
 F1-score 
 
 
 
 
 bert-base 
 0.832428 
 0.590338 
 0.445536 
 
 
 bert-pos-bow 
 0.828371 
 0.593237 
 0.405761 
 
 
 bert-pos-ner-bow 
 0.828184 
 0.590821 
 0.412519 
 
 
 bert-pos-tfidf 
 0.826166 
 0.597585 
 0.418983 
 
 
 bert-pos-ner-tfidf 
 0.827540 
 0.588889 
 0.411790 
 
 
 
 
 
 
 
   
 Loss 
 Accuracy 
 F1-score 
 
 
 
 
 bert-base 
 0.764947 
 0.673913 
 0.459577 
 
 
 bert-pos-bow 
 0.769999 
 0.683092 
 0.417592 
 
 
 bert-pos-ner-bow 
 0.778128 
 0.681643 
 0.406532 
 
 
 bert-pos-tfidf 
 0.764610 
 0.677778 
 0.470799 
 
 
 bert-pos-ner-tfidf 
 0.768630 
 0.676328 
 0.467827